# Maximally Advanced Pain Intensity Prediction Pipeline
This notebook implements a state-of-the-art, strategy-driven pipeline for pain intensity prediction from biosignals (EMG, SCL, ECG). It includes robust denoising, comprehensive feature engineering (including HRV, SCR, Poincaré, cross-channel, and relative features), per-segment normalization, class balancing, stacking ensemble, model-based feature selection, domain adaptation, interpretability (SHAP), and robust evaluation (learning/ROC curves, LOSO CV).

In [ ]:
# --- Imports and Setup ---
import os
import numpy as np
import pandas as pd
import scipy.io
from scipy.signal import butter, filtfilt, iirnotch, resample, find_peaks
from scipy.stats import skew, kurtosis, entropy
import pywt
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score, roc_curve, auc
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV, learning_curve
from sklearn.feature_selection import SelectFromModel
from imblearn.over_sampling import SMOTE
import shap
import matplotlib.pyplot as plt
from tqdm import tqdm

# Optional: Domain adaptation (CORAL)
def coral(Xs, Xt):
    # CORAL: Aligns source and target covariances
    cov_src = np.cov(Xs, rowvar=False) + np.eye(Xs.shape[1])
    cov_tar = np.cov(Xt, rowvar=False) + np.eye(Xt.shape[1])
    U_src, S_src, _ = np.linalg.svd(cov_src)
    U_tar, S_tar, _ = np.linalg.svd(cov_tar)
    A_src = U_src @ np.diag(1.0/np.sqrt(S_src)) @ U_src.T
    A_tar = U_tar @ np.diag(np.sqrt(S_tar)) @ U_tar.T
    return (Xs - Xs.mean(0)) @ A_src @ A_tar + Xt.mean(0)

# --- Signal Processing Utilities ---
def bandpass_filter(data, fs, lowcut, highcut, order=4):
    nyq = 0.5 * fs
    low, high = lowcut/nyq, highcut/nyq
    b, a = butter(order, [low, high], btype='band')
    return filtfilt(b, a, data, axis=0)

def notch_filter(data, fs, freq=50.0, Q=30):
    nyq = 0.5 * fs
    w0 = freq / nyq
    b, a = iirnotch(w0, Q)
    return filtfilt(b, a, data, axis=0)

def wavelet_denoise(sig, wavelet='db4', level=2):
    coeffs = pywt.wavedec(sig, wavelet, level=level)
    sigma = np.median(np.abs(coeffs[-level])) / 0.6745
    uthresh = sigma * np.sqrt(2*np.log(len(sig)))
    denoised_coeffs = [coeffs[0]] + [pywt.threshold(c, value=uthresh, mode='soft') for c in coeffs[1:]]
    return pywt.waverec(denoised_coeffs, wavelet)[:len(sig)]

# Optional: ICA for EMG artifact rejection (if sklearn.decomposition.FastICA is available)
def try_ica(sig, n_components=1):
    try:
        from sklearn.decomposition import FastICA
        ica = FastICA(n_components=n_components, random_state=42, max_iter=200)
        sig_ica = ica.fit_transform(sig.reshape(-1,1)).flatten()
        return sig_ica
    except Exception:
        return sig

def zscore_log_transform(x):
    x = np.array(x, dtype=float)
    x = np.log1p(np.abs(x)) * np.sign(x)
    mu, sigma = np.mean(x), np.std(x)
    return (x - mu) / sigma if sigma > 0 else np.zeros_like(x)

# --- HRV & Poincaré Features for ECG ---
def hrv_features(sig, fs):
    sig = np.nan_to_num(sig)
    peaks, _ = find_peaks(sig, distance=fs*0.5)
    rr = np.diff(peaks)/fs if len(peaks) > 1 else np.array([0])
    hr = 60/rr.mean() if rr.mean() > 0 else 0
    rmssd = np.sqrt(np.mean(np.square(np.diff(rr)))) if len(rr) > 1 else 0
    sdnn = rr.std() if len(rr) > 1 else 0
    # Frequency domain
    if len(rr) > 2:
        rr_interp = np.interp(np.arange(0, len(rr)), np.arange(0, len(rr)), rr)
        freqs = np.fft.rfftfreq(len(rr_interp), 1/fs)
        psd = np.abs(np.fft.rfft(rr_interp))**2
        lf = psd[(freqs >= 0.04) & (freqs < 0.15)].sum()
        hf = psd[(freqs >= 0.15) & (freqs < 0.4)].sum()
        lf_hf = np.log1p(lf/hf) if hf > 0 else 0
    else:
        lf = hf = lf_hf = 0
    # Poincaré
    sd1 = np.sqrt(np.std(rr[:-1] - rr[1:])**2/2) if len(rr) > 2 else 0
    sd2 = np.sqrt(2*np.std(rr)**2 - 0.5*np.std(rr[:-1] - rr[1:])**2) if len(rr) > 2 else 0
    return [hr, rmssd, sdnn, lf, hf, lf_hf, sd1, sd2]

# --- SCR Features for SCL ---
def scr_features(sig, fs):
    # Find SCRs as rapid increases in SCL
    diff = np.diff(sig)
    scr_peaks, _ = find_peaks(diff, height=np.std(diff), distance=fs*0.5)
    n_scr = len(scr_peaks)
    scr_amp = np.mean(diff[scr_peaks]) if n_scr > 0 else 0
    auc = np.trapz(sig)
    slope = (sig[-1] - sig[0]) / len(sig)
    return [n_scr, scr_amp, auc, slope]

# --- Feature Extraction (Comprehensive) ---
def extract_all_features(data, fs, ch_names):
    feats = []
    for idx, kind in enumerate(['EMG','EMG','EMG','SCL','ECG']):
        sig = data[:, idx]
        # Filtering and denoising
        if kind == 'EMG':
            sig = bandpass_filter(sig, fs, 20, 450)
            sig = notch_filter(sig, fs, 50)
            sig = try_ica(sig)
        elif kind == 'ECG':
            sig = bandpass_filter(sig, fs, 0.5, 50)
            sig = notch_filter(sig, fs, 50)
        elif kind == 'SCL':
            sig = bandpass_filter(sig, fs, 0.05, 5)
        sig = resample(sig, int(len(sig)//(fs/250))) if fs > 250 else sig
        sig = wavelet_denoise(sig)
        # Time-domain
        rms = np.sqrt(np.mean(sig**2))
        log_rms = np.log1p(rms)
        std = np.std(sig)
        wl = np.sum(np.abs(np.diff(sig)))
        zcr = ((sig[:-1]*sig[1:])<0).sum() / len(sig)
        feats.extend([rms, log_rms, std, wl, zcr, skew(sig), kurtosis(sig)])
        # Frequency-domain
        freqs = np.fft.rfftfreq(len(sig), 1/250)
        psd = np.abs(np.fft.rfft(sig))**2
        feats.append(np.sum(psd[(freqs>=0.5)&(freqs<4)]))
        feats.append(np.sum(psd[(freqs>=4)&(freqs<8)]))
        feats.append(np.sum(psd[(freqs>=8)&(freqs<13)]))
        feats.append(np.sum(psd[(freqs>=13)&(freqs<30)]))
        feats.append(np.sum(psd[(freqs>=30)&(freqs<45)]))
        # Wavelet energy/entropy
        coeffs = pywt.wavedec(sig, 'db4', level=3)
        energies = [np.sum(np.square(c)) for c in coeffs]
        total_energy = np.sum(energies)
        ent = -np.sum([(e/total_energy)*np.log(e/total_energy+1e-8) if e>0 else 0 for e in energies])
        feats.extend(energies + [ent])
        # ECG HRV/Poincaré
        if kind == 'ECG':
            feats.extend(hrv_features(sig, 250))
        # SCL SCR
        if kind == 'SCL':
            feats.extend(scr_features(sig, 250))
    # Cross-channel features (correlations, ratios)
    for i in range(data.shape[1]):
        for j in range(i+1, data.shape[1]):
            feats.append(np.corrcoef(data[:,i], data[:,j])[0,1])
            feats.append(np.mean(data[:,i]) / (np.mean(data[:,j])+1e-8))
    # Delta features (change from start to end)
    for idx in range(data.shape[1]):
        sig = data[:, idx]
        feats.append(sig[-1] - sig[0])
    return np.nan_to_num(feats)

# --- Data Loader ---
def collect_segments_all(root_dir):
    segments, labels, subjects, pain_map = [], [], [], {}
    for pain_label in sorted(os.listdir(root_dir)):
        pain_path = os.path.join(root_dir, pain_label)
        if not os.path.isdir(pain_path): continue
        for subj in sorted(os.listdir(pain_path)):
            subj_path = os.path.join(pain_path, subj)
            if not os.path.isdir(subj_path): continue
            for segfile in sorted(os.listdir(subj_path)):
                if not segfile.endswith('.mat'): continue
                matf = os.path.join(subj_path, segfile)
                try:
                    mat = scipy.io.loadmat(matf)
                    data = mat['data']
                    fs = int(mat['fs'].ravel()[0])
                    ch_names = [str(s[0]) if isinstance(s, np.ndarray) else str(s) for s in mat['labels'].ravel()]
                    feats = extract_all_features(data, fs, ch_names)
                    segments.append(feats)
                    labels.append(pain_label)
                    subjects.append(subj)
                    pain_map[pain_label] = pain_map.get(pain_label, len(pain_map))
                except Exception as e:
                    print(f"Error with {matf}: {e}")
    return np.array(segments), np.array(labels), np.array(subjects), pain_map

# --- Main Pipeline ---
root_dir = r"D:\bio_s"  # Change if needed
print("Loading and extracting all advanced features...")
X, y, subjects, pain_map = collect_segments_all(root_dir)
if len(X) == 0:
    raise RuntimeError("No valid segments loaded. Please check your data and preprocessing functions.")
print(f"Total segments: {len(X)}, Features per segment: {X.shape[1]}")
label_names = sorted(pain_map, key=lambda k: pain_map[k])
y_enc = np.array([pain_map[lab] for lab in y])

# --- Per-segment normalization and log transform ---
X = np.apply_along_axis(zscore_log_transform, 1, X)

# --- Class Balancing ---
sm = SMOTE(random_state=42)
X_bal, y_bal = sm.fit_resample(X, y_enc)

# --- Model-based Feature Selection ---
sel = SelectFromModel(RandomForestClassifier(n_estimators=100, random_state=42), threshold='median')
sel.fit(X_bal, y_bal)
X_sel = sel.transform(X_bal)

# --- Train/Test Split ---
X_train, X_test, y_train, y_test = train_test_split(X_sel, y_bal, test_size=0.3, stratify=y_bal, random_state=42)

# --- Domain Adaptation (CORAL) ---
try:
    X_train = coral(X_train, X_test)
except Exception as e:
    print(f"CORAL domain adaptation skipped: {e}")

# --- Stacking Ensemble (XGB, LGBM, SVM, meta-learner) ---
base_learners = [
    ('xgb', XGBClassifier(n_estimators=200, max_depth=6, eval_metric='mlogloss', use_label_encoder=False, random_state=42)),
    ('lgbm', LGBMClassifier(n_estimators=200, max_depth=6, class_weight='balanced', random_state=42)),
    ('svm', SVC(kernel='rbf', probability=True, class_weight='balanced', random_state=42)),
]
meta_learner = RandomForestClassifier(n_estimators=100, random_state=42)
stack = StackingClassifier(estimators=base_learners, final_estimator=meta_learner, cv=3, n_jobs=-1, passthrough=True)

# --- Hyperparameter Tuning ---
param_grid = {
    'xgb__n_estimators': [100, 200],
    'xgb__max_depth': [4, 6],
    'lgbm__n_estimators': [100, 200],
    'lgbm__max_depth': [4, 6],
    'svm__C': [1, 10],
    'svm__gamma': ['scale', 'auto'],
    'final_estimator__max_depth': [4, 8],
}
grid = GridSearchCV(stack, param_grid, cv=3, scoring='accuracy', n_jobs=-1)
grid.fit(X_train, y_train)

# --- Evaluation ---
y_pred = grid.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {acc:.3f}")
print(classification_report(y_test, y_pred, target_names=label_names))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# --- Learning Curve ---
train_sizes, train_scores, test_scores = learning_curve(grid.best_estimator_, X_train, y_train, cv=3, n_jobs=-1)
plt.figure()
plt.plot(train_sizes, np.mean(train_scores, axis=1), label='Train')
plt.plot(train_sizes, np.mean(test_scores, axis=1), label='Validation')
plt.xlabel('Training Size')
plt.ylabel('Accuracy')
plt.title('Learning Curve')
plt.legend()
plt.show()

# --- ROC Curve (One-vs-Rest) ---
from sklearn.preprocessing import label_binarize
n_classes = len(np.unique(y_test))
y_test_bin = label_binarize(y_test, classes=range(n_classes))
y_score = grid.predict_proba(X_test)
fpr, tpr, roc_auc = dict(), dict(), dict()
plt.figure()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
    plt.plot(fpr[i], tpr[i], label=f'Class {label_names[i]} (AUC={roc_auc[i]:.2f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve (One-vs-Rest)')
plt.legend()
plt.show()

# --- SHAP Feature Importance (XGB, LGBM) ---
for name in ['xgb', 'lgbm']:
    try:
        model = grid.best_estimator_.named_estimators_[name]
        explainer = shap.TreeExplainer(model)
        shap_values = explainer.shap_values(X_test)
        shap.summary_plot(shap_values, X_test, feature_names=[f'f{i}' for i in range(X_test.shape[1])], show=False)
        plt.title(f'SHAP Feature Importance ({name.upper()})')
        plt.show()
    except Exception as e:
        print(f"SHAP plot error for {name}: {e}")

# --- LOSO CV for Robustness ---
accs = []
all_true = []
all_pred = []
for subj in np.unique(subjects):
    test_idx = (subjects == subj)
    train_idx = ~test_idx
    if np.sum(test_idx) == 0 or np.sum(train_idx) == 0:
        continue
    Xtr, Xte = X[train_idx], X[test_idx]
    ytr, yte = y_enc[train_idx], y_enc[test_idx]
    Xtr_bal, ytr_bal = sm.fit_resample(Xtr, ytr)
    Xtr_sel = sel.transform(Xtr_bal)
    Xte_sel = sel.transform(Xte)
    try:
        Xtr_sel = coral(Xtr_sel, Xte_sel)
    except Exception:
        pass
    model = grid.best_estimator_
    model.fit(Xtr_sel, ytr_bal)
    ypred = model.predict(Xte_sel)
    accs.append(accuracy_score(yte, ypred))
    all_true.extend(yte)
    all_pred.extend(ypred)
print(f"\nLOSO Mean Accuracy: {np.mean(accs):.3f}")
print("\nLOSO Classification Report:")
print(classification_report(all_true, all_pred, target_names=label_names))
print("LOSO Confusion Matrix:")
print(confusion_matrix(all_true, all_pred))

**Summary:**
- This pipeline uses robust denoising, comprehensive feature engineering (including HRV, SCR, Poincaré, cross-channel, and relative features), per-segment normalization, class balancing, model-based feature selection, domain adaptation, stacking ensemble, and interpretability (SHAP).
- It provides learning/ROC curves and robust LOSO CV evaluation for generalization.
- Tune the feature set or model parameters further if needed to maximize accuracy and recall.